In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/convolve-epoch1/sample_submission.csv
/kaggle/input/convolve-epoch1/train.json
/kaggle/input/convolve-epoch1/test.csv


In [1]:
pip install tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 26.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 60.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 58.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.8 MB/s eta 0:00:00:00:01
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np
import pandas as pd

2022-12-16 08:17:46.187074: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-16 08:17:47.552113: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-12-16 08:17:47.552332: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Co

In [4]:
import json
with open('/kaggle/input/convolve-epoch1/train.json','r') as f:
    data=json.load(f)
    
train = pd.DataFrame({'Label':data})

In [5]:
arr1=np.array(train.index)
arr2=np.array(train['Label'])
train=pd.DataFrame()
train['Log']=arr1
train['Label']=arr2
train

,Log,Label
0,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal
1,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal
2,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal
3,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal
4,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal
...,...,...
4152654,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal
4152655,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal
4152656,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal
4152657,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal


In [7]:
train['Label'].value_counts()

normal      4082967
abnormal      69692
Name: Label, dtype: int64

In [8]:
train_abnormal = train[train['Label']=='abnormal']
train_abnormal

,Log,Label
4082967,1117869872 2005.06.04 R04-M1-N4-I:J18-U01 2005...,abnormal
4082968,1117869872 2005.06.04 R04-M1-N4-I:J18-U11 2005...,abnormal
4082969,1117869872 2005.06.04 R23-M1-N8-I:J18-U11 2005...,abnormal
4082970,1117869873 2005.06.04 R22-M1-N8-I:J18-U01 2005...,abnormal
4082971,1117869873 2005.06.04 R22-M1-N8-I:J18-U11 2005...,abnormal
...,...,...
4152654,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal
4152655,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal
4152656,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal
4152657,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal


In [9]:
train_normal = train[train['Label']=='normal']
train_normal.shape

(4082967, 2)

In [10]:
train_normal_downsampled = train_normal.sample(train_abnormal.shape[0])
train_normal_downsampled.shape

(69692, 2)

In [11]:
train_balanced = pd.concat([train_normal_downsampled, train_abnormal])
train_balanced

,Log,Label
3404213,1131057326 2005.11.03 R63-M0-NE-C:J10-U11 200...,normal
2912244,1125084231 2005.08.26 R70-M1-N4-C:J13-U01 200...,normal
2069187,1121496251 2005.07.15 R07-M0-N6-C:J14-U01 200...,normal
1023720,1120216078 2005.07.01 R30-M1-N4-C:J06-U01 200...,normal
2409562,1122166534 2005.07.23 R10-M1-N6-C:J13-U11 200...,normal
...,...,...
4152654,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal
4152655,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal
4152656,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal
4152657,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal


In [12]:
train_balanced.reset_index(inplace=True)
train_balanced

,index,Log,Label
0,3404213,1131057326 2005.11.03 R63-M0-NE-C:J10-U11 200...,normal
1,2912244,1125084231 2005.08.26 R70-M1-N4-C:J13-U01 200...,normal
2,2069187,1121496251 2005.07.15 R07-M0-N6-C:J14-U01 200...,normal
3,1023720,1120216078 2005.07.01 R30-M1-N4-C:J06-U01 200...,normal
4,2409562,1122166534 2005.07.23 R10-M1-N6-C:J13-U11 200...,normal
...,...,...,...
139379,4152654,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal
139380,4152655,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal
139381,4152656,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal
139382,4152657,1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal


In [13]:
train_balanced['normal']=train_balanced['Label'].apply(lambda x:1 if x=='normal' else 0)
train_balanced.sample(5)

,index,Log,Label,normal
30589,479469,1118772326 2005.06.14 R27-M1-NF-C:J13-U01 200...,normal,1
119512,4132787,1118542363 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal,0
105477,4118752,1118540266 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal,0
21256,261269,1118765823 2005.06.14 R11-M0-N5-C:J02-U01 200...,normal,1
126367,4139642,1118543476 2005.06.11 R30-M0-N9-C:J16-U01 2005...,abnormal,0


In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_balanced['Log'],train_balanced['normal'],stratify=train_balanced['normal'])

In [15]:
X_train.sample(5)

80653     1118536658 2005.06.11 R30-M0-N9-C:J16-U01 2005...
128763    1118543840 2005.06.11 R30-M0-N9-C:J16-U01 2005...
69015      1120956728 2005.07.09 R34-M1-N0-C:J09-U01 200...
134121    1118544668 2005.06.11 R30-M0-N9-C:J16-U01 2005...
43705      1127243399 2005.09.20 NULL 2005-09-20-12.09.5...
Name: Log, dtype: object

In [16]:
y_train.sample(5)

63127     1
72009     0
33416     1
107624    0
29204     1
Name: normal, dtype: int64

In [17]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [19]:
#Layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(2, activation = 'tanh')(l)
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [21]:
len(X_train)

104538

In [25]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [26]:
model.fit(X_train, y_train, epochs=1)

3267/3267 [==============================] - 1114s 340ms/step - loss: 0.0276 - accuracy: 0.9914 - precision: 0.9878 - recall: 0.9951


In [27]:
#this is to save the model weights and checkouot
model.save_weights('./kaggle/working/convolve_model_weights')

In [28]:
model.evaluate(X_test, y_test)

1089/1089 [==============================] - 372s 340ms/step - loss: 0.0095 - accuracy: 0.9988 - precision: 0.9985 - recall: 0.9991


[0.009464756585657597,
 0.9988234043121338,
 0.998508632183075,
 0.9991390705108643]

In [30]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

1089/1089 [==============================] - 368s 338ms/step


array([1, 0, 1, ..., 1, 1, 1])

In [31]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, y_predicted)
cm 

array([[17397,    26],
       [   15, 17408]])

In [32]:
test_data=pd.read_csv("/kaggle/input/convolve-epoch1/test.csv")
test_data

,ID,Log
0,0,1124336301 2005.08.17 R13-M1-N8-C:J12-U01 200...
1,1,1118553175 2005.06.11 R30-M0-N9-C:J16-U01 200...
2,2,1118536033 2005.06.11 R30-M0-N9-C:J16-U01 200...
3,3,1117992566 2005.06.05 R30-M1-N6-C:J03-U11 200...
4,4,1118538965 2005.06.11 R30-M0-N9-C:J16-U01 200...
...,...,...
595295,595295,1118768234 2005.06.14 R20-M0-NA-C:J17-U11 200...
595296,595296,1118370947 2005.06.09 R15-M0-NA-C:J02-U01 200...
595297,595297,1118540242 2005.06.11 R30-M0-N9-C:J16-U01 200...
595298,595298,1118538947 2005.06.11 R30-M0-N9-C:J16-U01 200...


In [33]:
X_test = np.array(test_data[' Log'])
len(X_test)

595300

In [37]:
predictions = model.predict(X_test)


18604/18604 [==============================] - 6345s 341ms/step


In [41]:
y_predicted = predictions.flatten()
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([1, 0, 0, ..., 0, 0, 1])

In [42]:
len(y_predicted)

595300

In [43]:
submission=pd.read_csv('/kaggle/input/convolve-epoch1/sample_submission.csv')
submission

,ID,Label
0,0,normal
1,1,normal
2,2,normal
3,3,normal
4,4,normal
...,...,...
595295,595295,normal
595296,595296,normal
595297,595297,normal
595298,595298,normal


In [44]:
submission['Label']=y_predicted

In [45]:
submission

,ID,Label,Label
0,0,normal,1
1,1,normal,0
2,2,normal,0
3,3,normal,1
4,4,normal,0
...,...,...,...
595295,595295,normal,1
595296,595296,normal,1
595297,595297,normal,0
595298,595298,normal,0


In [46]:
submission.columns

Index(['ID', ' Label', 'Label'], dtype='object')

In [47]:
submission.drop(' Label',axis=1,inplace=True)
submission

,ID,Label
0,0,1
1,1,0
2,2,0
3,3,1
4,4,0
...,...,...
595295,595295,1
595296,595296,1
595297,595297,0
595298,595298,0


In [49]:
submission['Label']=submission['Label'].apply(lambda x: 'normal' if x==1 else 'abnormal')
submission

,ID,Label
0,0,normal
1,1,abnormal
2,2,abnormal
3,3,normal
4,4,abnormal
...,...,...
595295,595295,normal
595296,595296,normal
595297,595297,abnormal
595298,595298,abnormal


In [50]:
submission.rename(columns = {'Label':' Label'}, inplace = True)
submission

,ID,Label
0,0,normal
1,1,abnormal
2,2,abnormal
3,3,normal
4,4,abnormal
...,...,...
595295,595295,normal
595296,595296,normal
595297,595297,abnormal
595298,595298,abnormal


In [51]:
submission.columns


Index(['ID', ' Label'], dtype='object')

In [52]:
submission[' Label'].value_counts()

normal      368859
abnormal    226441
Name:  Label, dtype: int64

In [53]:
submission.to_csv('submission.csv',index=False)